<a href="https://colab.research.google.com/github/saibhargavinukala/cryptography/blob/main/Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Columnar Transportation

In [ ]:
def ct_encrypt(text, key):
    columns = len(key)
    while len(text) % columns != 0:
        text += '_'
    grid = ['' for _ in range(columns)]
    for i, char in enumerate(text):
        grid[i % columns] += char
    sorted_key = sorted((k, i) for i, k in enumerate(key))
    result = ''.join(grid[index] for _, index in sorted_key)
    return result

def ct_decrypt(ciphertext, key):
    columns = len(key)
    rows = len(ciphertext) // columns
    grid = ['' for _ in range(columns)]
    sorted_key = sorted((k, i) for i, k in enumerate(key))
    idx = 0
    for _, index in sorted_key:
        grid[index] = ciphertext[idx:idx + rows]
        idx += rows
    result = ''.join(grid[i % columns][i // columns] for i in range(len(ciphertext)))
    result = result.rstrip('_')
    return result

plaintext = "cryptography"
key = "51324"
encrypted_text = ct_encrypt(plaintext, key)
decrypted_text = ct_decrypt(encrypted_text, key)

print(f"Encrypted: {encrypted_text}")
print(f"Decrypted: {decrypted_text}")


Encrypted: rgypa_yr_tp_coh
Decrypted: cryptography


Ceaser Cipher

In [ ]:
def caesar_cipher(msg, shift):
    result = ""
    for char in msg:
        if char.isalpha():
            shift_base = 65 if char.isupper() else 97
            result += chr((ord(char) - shift_base + shift) % 26 + shift_base)
        else:
            result += char
    return result

def caesar_decipher(ciphertext, shift):
    return caesar_cipher(ciphertext, -shift)

plaintext = "HELLO WORLD"
shift = 3
encrypted_text = caesar_cipher(plaintext, shift)
decrypted_text = caesar_decipher(encrypted_text, shift)

print(f"Encrypted: {encrypted_text}")
print(f"Decrypted: {decrypted_text}")


Encrypted: KHOOR ZRUOG
Decrypted: HELLO WORLD


AES

In [ ]:
! pip install pycryptodome

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def aes_encrypt(text, key):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_text = pad(text.encode(), AES.block_size)
    return cipher.encrypt(padded_text)

def aes_decrypt(encrypted_text, key):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_text = unpad(cipher.decrypt(encrypted_text), AES.block_size)
    return decrypted_text.decode()

key = b'sixteen byte key'
text = "Hello, AES!"
encrypted_text = aes_encrypt(text, key)
decrypted_text = aes_decrypt(encrypted_text, key)

print(f"Encrypted: {encrypted_text}")
print(f": {decrypted_text}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.7 MB/s eta 0:00:00
Encrypted: b'\xe4Q\x8d\xb1\xe6\x90\x81\xe0\t\x85\xc3.j\xe6Y\x95'
: Hello, AES!


PlayFair Cipher

In [ ]:
def playfair_cipher(text, key):
    text = text.lower().replace(" ", "")
    if len(text) % 2 != 0:
        text += 'x'

    diagraphs = [text[i:i + 2] for i in range(0, len(text), 2)]
    alphabet = 'abcdefghiklmnopqrstuvwxyz'
    key = key.lower()
    key = ''.join(sorted(set(key), key=lambda x: key.index(x)))
    key += ''.join(sorted(set(alphabet) - set(key)))
    key_matrix = [list(key[i:i + 5]) for i in range(0, len(key), 5)]

    def search(matrix, char):
        for row in range(5):
            if char in matrix[row]:
                return row, matrix[row].index(char)
        return None

    def encrypt_pair(matrix, char1, char2):
        r1, c1 = search(matrix, char1)
        r2, c2 = search(matrix, char2)
        if r1 == r2:
            return matrix[r1][(c1 + 1) % 5] + matrix[r1][(c2 + 1) % 5]
        elif c1 == c2:
            return matrix[(r1 + 1) % 5][c1] + matrix[(r2 + 1) % 5][c2]
        else:
            return matrix[r1][c2] + matrix[r2][c1]

    def decrypt_pair(matrix, char1, char2):
        r1, c1 = search(matrix, char1)
        r2, c2 = search(matrix, char2)
        if r1 == r2:
            return matrix[r1][(c1 - 1) % 5] + matrix[r1][(c2 - 1) % 5]
        elif c1 == c2:
            return matrix[(r1 - 1) % 5][c1] + matrix[(r2 - 1) % 5][c2]
        else:
            return matrix[r1][c2] + matrix[r2][c1]

    encrypted_text = ''.join(encrypt_pair(key_matrix, pair[0], pair[1]) for pair in diagraphs)
    decrypted_diagraphs = [encrypted_text[i:i + 2] for i in range(0, len(encrypted_text), 2)]
    decrypted_text = ''.join(decrypt_pair(key_matrix, pair[0], pair[1]) for pair in decrypted_diagraphs)

    if decrypted_text.endswith('x') and len(decrypted_text) > 1:
        decrypted_text = decrypted_text[:-1]

    return encrypted_text, decrypted_text

key = "Monarchy"
text = "instruments"
encrypted_text, decrypted_text = playfair_cipher(text, key)

print(f"Encrypted: {encrypted_text}")
print(f"Decrypted: {decrypted_text}")


Encrypted: gatlmzclrqxa
Decrypted: instruments


RSA

In [ ]:
p, q = 61,53
n = p * q
phi = (p - 1) * (q - 1)
e = 5
from math import gcd
while gcd(e, phi) != 1: e += 2
d = pow(e, -1, phi)
message = 42
ciphertext = pow(message, e, n)
plaintext = pow(ciphertext, d, n)
print(f"Public Key: ({e}, {n})\nPrivate Key: ({d}, {n})")
print(f"Encrypted Message: {ciphertext}\nDecrypted Message: {plaintext}")

Public Key: (7, 3233)
Private Key: (1783, 3233)
7 3233
1783 3233
Encrypted Message: 240
Decrypted Message: 42


Diifie - Hellman


In [ ]:
def power(a, b, p):
  return pow(a, b, p)
P, G, a, b = 23, 9, 4, 3
x, y = power(G, a, P), power(G, b, P)
print("Secret keys:", power(y, a, P), power(x, b, P))
print("Private keys:", a, b)

Secret keys: 9 9
Private keys: 4 3


RC4

In [ ]:
def rc4(key, plaintext):
    S = list(range(256))
    j = 0
    for i in range(256):
        w =  ord(key[i % len(key)])
        j = (j + S[i] + w) % 256
        S[i], S[j] = S[j], S[i]
    i = j = 0
    keystream = []
    for _ in range(len(plaintext)):
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        keystream.append(S[(S[i] + S[j]) % 256])
    result = []
    for c, k in zip(plaintext, keystream):
        result.append(chr(ord(c) ^ k))
    return ''.join(result)
key = "simplekey"
plaintext = "Hello,RC4!"
ciphertext = rc4(key, plaintext)
print("Ciphertext:", ciphertext)
decrypted = rc4(key, ciphertext)
print("Decrypted:", decrypted)

Ciphertext: ÂJö,SìU
Decrypted: Hello,RC4!


SHA-3


In [ ]:
import hashlib
def simple_sha3(input_string):
    sha3_hash = hashlib.sha3_256()
    sha3_hash.update(input_string.encode('utf-8'))
    return sha3_hash.hexdigest()
input_text = "Hello, SHA-3!"
hashed_output = simple_sha3(input_text)
print("Input:", input_text)
print("SHA-3 Hash:", hashed_output)

Input: Hello, SHA-3!
SHA-3 Hash: ce4ab23ba79eba9ba2531220a647080bed52619b752df89a0a67fd5973d667f1


HMAC

In [ ]:
import hmac
import hashlib
def generate_hmac(key, message):
    hmac_obj = hmac.new(key.encode(), message.encode(), hashlib.sha256)
    return hmac_obj.hexdigest()
key = "48"
message = "Hello, HMAC!"
hmac_output = generate_hmac(key, message)
print("Message:", message)
print("HMAC:", hmac_output)

Message: Hello, HMAC!
HMAC: 94ad79e245f7b3b0bc3fa1c20dece33f3cfc02777a36d58f164e407506369639


DSS


In [ ]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes
private_key = dsa.generate_private_key(key_size=2048)
public_key = private_key.public_key()
message = b"Hello, DSS!"

signature = private_key.sign(
    message,
    hashes.SHA256()
)

print("Digital Signature:", signature)

try:
    public_key.verify(
        signature,
        message,
        hashes.SHA256()
    )
    print("Signature is valid.")
except Exception as e:
    print("Signature verification failed:", str(e))

Digital Signature: b'0F\x02!\x00\xc3\x8f\xeaV>\xbbF\x8b:\x1b\x93\xc7\xc5\x04\xdd\x93\xc0\x07\x0c\xeex\xbd\x94-\xf9t\xfc\x05\xf8\\\xaf"\x02!\x00\x85R\x0b\x17GO\n\x8b\xaa~G\xd1\x04\x1eg\x8f\x1e\x16o\x08Ux6\xb5e\xe0\xbd\ts\x10GX'
Signature is valid.
